## Role: Hospital-Patient

### This notebook acts like an issuer in blockchain. What we do is to generate a patient did and write them to buildernet. Furthermore, it can also issue personal data to patients' wallet to protect their private data from hackers.


### 1. Initialise a controller for Hospital Agent

In [1]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


In [2]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8072
WEBHOOK_BASE = ""
ADMIN_URL = "http://hospital-agent:8071"

# WARNING: You should use environment variables for this
# TODO: Make env variables accessible through juypter notebooks
# API_KEY = "alice_api_123456789"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL)

In [3]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST,
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)


### 3. Generate a new DID for this visit


Before being able to write a DID to the ledger, you must create one using the wallet api. This api returns the identifier (the DID), and the verification key for that DID. A representation of it's public key.



In [7]:
# generate new DID
response = await agent_controller.wallet.create_did()

did_object = response['result']
print("New DID", did_object)

New DID {'did': 'ANQVozG2Ex2nS15xkHW6LL', 'verkey': '67B7Csa6tcGGCVTCidzLTWRAzYVQvgphZvuyjQ4S9hJG', 'posture': 'wallet_only'}


### 4. Write DID to Ledger

Anoyone can write a DID to the Sovrin Buildernet, it is a permissionless ledger.

Visit Sovrin Selfserve Portal for more information. We have provided an automated process to write DIDs to Buildernet in the step below.

In [8]:
result = await agent_controller.ledger.register_nym(did_object['did'], did_object['verkey'])
print(result)

{'success': True}


### 6. Set public DID


Now you are able to assign the DID written to the ledger as public.



In [9]:
response = await agent_controller.wallet.assign_public_did(did_object["did"])
print(response)

{'result': {'did': 'ANQVozG2Ex2nS15xkHW6LL', 'verkey': '67B7Csa6tcGGCVTCidzLTWRAzYVQvgphZvuyjQ4S9hJG', 'posture': 'public'}}


### 7. Get public DID

In [4]:
response = await agent_controller.wallet.get_public_did()
print(response)
issuer_nym = response['result']['did']
print('\nIssuer public DID:',issuer_nym)

{'result': {'did': 'PEcmuSQ7G3z771J7dse1R5', 'verkey': 'D7tL6gtUUqSD4wF1e4rVcrVqxQoo65EgKquja9qeN9Mx', 'posture': 'public'}}

Issuer public DID: PEcmuSQ7G3z771J7dse1R5


### 8. Fetch verkey for public DID

Additionally, we can verify that this DID does actually resolve to the public key material on the ledger.



In [11]:
issuer_verkey = await agent_controller.ledger.get_did_verkey(issuer_nym)
print(issuer_verkey)

{'verkey': '67B7Csa6tcGGCVTCidzLTWRAzYVQvgphZvuyjQ4S9hJG'}


### 9. Get public DID endpoint


As well as providing a publically accessible endpoint to contact the DID controller through the agent framework.



In [12]:
issuer_endpoint = await agent_controller.ledger.get_did_endpoint(issuer_nym)
print(issuer_endpoint)

{'endpoint': 'http://192.168.65.9:8070'}


### 2. Register Listeners

The handler should get called every time the controller receives a webhook with the topic issue_credential, printing out the payload. The agent calls to this webhook every time it receives an issue-credential protocol message from a credential.

In [5]:
response = await agent_controller.wallet.assign_public_did("PEcmuSQ7G3z771J7dse1R5")

print(response)

{'result': {'did': 'PEcmuSQ7G3z771J7dse1R5', 'verkey': 'D7tL6gtUUqSD4wF1e4rVcrVqxQoo65EgKquja9qeN9Mx', 'posture': 'public'}}


In [4]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Offering: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}

def connections_handler(payload):
    global STATE
    connection_id = payload["connection_id"]
    print("Connection message", payload, connection_id)
    STATE = payload['state']
    if STATE == 'active':
#         print('Connection {0} changed state to active'.format(connection_id))
        print(colored("Connection {0} changed state to active".format(connection_id), "red", attrs=["bold"]))


connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

agent_controller.register_listeners([cred_listener,connection_listener], defaults=True)

Subscribing too: issue_credential
Subscribing too: connections


### Check Connection

In [5]:
response = await agent_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
    print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
else:
    print("You must create a connection")
    

Results :  [{'accept': 'manual', 'routing_state': 'none', 'connection_id': 'cd328dc3-35e7-452a-9257-b0d98aa50ce8', 'their_role': 'inviter', 'created_at': '2023-01-05 08:30:24.814303Z', 'request_id': '9684c553-f9d1-4c24-aa23-8e78422e53ea', 'updated_at': '2023-01-05 08:30:40.077500Z', 'my_did': 'Gtk5VfRz9RxA2wCzLsPkgo', 'their_did': '6kaHyKtCrNToKDBtcHq3rF', 'their_label': 'THE_WALLET', 'invitation_key': 'TxozusBDkopmyRZuee8jPCSZQZmXpJEpVSvfD1U4xpS', 'rfc23_state': 'completed', 'state': 'active', 'invitation_mode': 'once'}]
Connection : {'accept': 'manual', 'routing_state': 'none', 'connection_id': 'cd328dc3-35e7-452a-9257-b0d98aa50ce8', 'their_role': 'inviter', 'created_at': '2023-01-05 08:30:24.814303Z', 'request_id': '9684c553-f9d1-4c24-aa23-8e78422e53ea', 'updated_at': '2023-01-05 08:30:40.077500Z', 'my_did': 'Gtk5VfRz9RxA2wCzLsPkgo', 'their_did': '6kaHyKtCrNToKDBtcHq3rF', 'their_label': 'THE_WALLET', 'invitation_key': 'TxozusBDkopmyRZuee8jPCSZQZmXpJEpVSvfD1U4xpS', 'rfc23_state': 'co

### 3b. (OPTIONAL) Write a Schema to the Ledger

For more details see the schema-api notebook

Note: You will only be able to do this once unless you change the schema_name or version. Once you have a schema written to the ledger it can be reused by multiple entities

In [14]:
# Define you schema name - must be unique on the ledger
schema_name = "MedicalRecords"
# Can version the schema if you wish to update it
schema_version = "0.0.9"
# Define any list of attributes you wish to include in your schema
attributes = ["fullname", "age", "date", "doctorname", "complaint", "examination results", "final treatment", "medicine", "nextappointment"]

response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
schema_id = response["schema_id"]
print(schema_id)

PEcmuSQ7G3z771J7dse1R5:2:MedicalRecords:0.0.9


### 4. Write a Credential Definition to the Ledger


Note: Again this can only be done once per issuer, per schema_id.



In [15]:
response = await agent_controller.definitions.write_cred_def(schema_id)

cred_def_id = response["credential_definition_id"]
print(cred_def_id)


PEcmuSQ7G3z771J7dse1R5:3:CL:40:default


### 5. Populate the Attribues to Issue to Idenity Holder (User)


We will issue a credential to the identity holder consisting of the following attributes:

credential_attributes = [ {"name": "fullname", "value": name}, {"name": "skill", "value": "PyDentity SSI Ninja"}, {"name": "age", "value": age} ]

The notebook will ask you to input the identity holder's full name and age which will be used to populate the schema above with the identity holders attribute information.

In [19]:
# from datetime import datetime

name=input("Please enter your name and surname: ")
age=input("Please enter your age: ")
TreatmentDate=input("Please enter treatment date: ")
doctor=input("Please enter doctor's name: ")
ChiefComplaint=input("Please enter chief complaint: ")
examination=input("Please enter examination: ")
treatment=input("Please enter treatment: ")
PrescriptionPad=input("Please enter prescription pad: ")
appointment=input("Please enter appointment: ")

credential_attributes = [
    {"name": "fullname", "value": name},
    {"name": "age", "value": age},
    {"name": "date", "value": TreatmentDate},
    {"name": "doctorname", "value": doctor},
    {"name": "complaint", "value": ChiefComplaint},
    {"name": "examination results", "value": examination},
    {"name": "final treatment", "value": treatment},
    {"name": "medicine", "value": PrescriptionPad},
    {"name": "nextappointment", "value": appointment}
]
print(credential_attributes)

Please enter your name and surname:  1
Please enter your age:  1
Please enter treatment date:  1
Please enter doctor's name:  1
Please enter chief complaint:  1
Please enter examination:  1
Please enter treatment:  1
Please enter prescription pad:  1
Please enter appointment:  1


[{'name': 'fullname', 'value': '1'}, {'name': 'age', 'value': '1'}, {'name': 'date', 'value': '1'}, {'name': 'doctorname', 'value': '1'}, {'name': 'complaint', 'value': '1'}, {'name': 'examination results', 'value': '1'}, {'name': 'final treatment', 'value': '1'}, {'name': 'medicine', 'value': '1'}, {'name': 'nextappointment', 'value': '1'}]


### Copy the invitation above and go to wallet notebook to accept accept connection

### 9. Send Credential

his sends a credential to a identity holder (User), and automates the rest of the protocol.

There are other ways to issue a credential that require multiple api calls.

Arguments

- connection_id: The connection_id of the holder you wish to issue to (MUST be in active state)
- schema_id: The id of the schema you wish to issue
- cred_def_id: The definition (public key) to sign the credential object. This must refer to the schema_id and be written to the ledger by the same public did that is currently being used by the agent.
- attributes: A list of attribute objects as defined above. Must match the schema attributes.
- comment (optional): Any string, defaults to ""
- auto_remove (optional): Boolean, defaults to True. I believe this removes the record of this credential once it has been issued. (TODO: double check)
- trace (optional): Boolean, defaults to False. Not entirely sure about this one, believe its for logging. Also when set to True it throws an error

In [20]:
record = await agent_controller.issuer.send_credential(connection_id, schema_id, cred_def_id, credential_attributes, trace=False)
record_id = record['credential_exchange_id']
state = record['state']
role = record['role']
print(f"Credential exchange {record_id}, role: {role}, state: {state}")

Credential exchange 4b707883-0dad-4365-b3c0-79eb51d00381, role: issuer, state: offer_sent
Handle Credentials
Credential exchange 4b707883-0dad-4365-b3c0-79eb51d00381, role: issuer, state: offer_sent
Offering: [{'name': 'fullname', 'value': '1'}, {'name': 'age', 'value': '1'}, {'name': 'date', 'value': '1'}, {'name': 'doctorname', 'value': '1'}, {'name': 'complaint', 'value': '1'}, {'name': 'examination results', 'value': '1'}, {'name': 'final treatment', 'value': '1'}, {'name': 'medicine', 'value': '1'}, {'name': 'nextappointment', 'value': '1'}]
Handle Credentials
Credential exchange 4b707883-0dad-4365-b3c0-79eb51d00381, role: issuer, state: request_received
Offering: [{'name': 'fullname', 'value': '1'}, {'name': 'age', 'value': '1'}, {'name': 'date', 'value': '1'}, {'name': 'doctorname', 'value': '1'}, {'name': 'complaint', 'value': '1'}, {'name': 'examination results', 'value': '1'}, {'name': 'final treatment', 'value': '1'}, {'name': 'medicine', 'value': '1'}, {'name': 'nextappoint

### 10. Accept credential in Mobile SSI Wallet


In [24]:
response = await agent_controller.terminate()
print(response)

None
